# Web Scraping Assignment 3 

### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

In [2]:
# Activating the chrome browser
driver = webdriver.Chrome('chromedriver.exe')

In [3]:
# Opening the homepage of Amazon.in
driver.get('https://www.amazon.in/')
# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : watch


In [4]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating search_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

#### Q2. 2.	In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.


In [6]:
prod_info = {}
prod_info['Brand']=[]
prod_info['Name']=[]
prod_info['Rating']=[]
prod_info['No. of ratings']=[]
prod_info['Price']=[]
prod_info['Return/Exchange']=[]
prod_info['Expected Delivery']=[] 
prod_info['Availability']=[]
prod_info['Other Details']=[]
prod_info['URL']=[]

In [7]:
for url in urls:
    # Loading the webpage by url
    driver.get(url)                                                        
    
    # Extracting Brand from xpath
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      
        prod_info['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_info['Brand'].append('-')
    
     # Extracting Name from xpath
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')     
        prod_info['Name'].append(name.text)
    except NoSuchElementException:
        prod_info['Name'].append('-')
    
    # Extracting Ratings from xpath
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  
        prod_info['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_info['Rating'].append('-')
    
     # Extracting no. of Ratings from xpath
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')    
        prod_info['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_info['No. of ratings'].append('-')
    
     # Extracting Price from xpath
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')           
        prod_info['Price'].append(price.text)
    except NoSuchElementException:
        prod_info['Price'].append('-')
        
    # Extracting Return/Exchange policy from xpath
    try:                                                                                     
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_info['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_info['Return/Exchange'].append('-')
    
     # Extracting Expected Delivery from xpath
    try:
        delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')        
        prod_info['Expected Delivery'].append(delivery.text)
    except NoSuchElementException:
        prod_info['Expected Delivery'].append('-')
    
    # Extracting Availability from xpath
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                
        prod_info['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_info['Availability'].append('-')
    
    # Extracting Other Details from xpath
    try:                                                                                    
        details = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_info['Other Details'].append('  ||  '.join(details.text.split('\n')))
    except NoSuchElementException:
        prod_info['Other Details'].append('-')
    
    # Extracting url
    prod_info['URL'].append(url)                                                           
    time.sleep(2)

In [8]:
products = pd.DataFrame.from_dict(prod_info)
products

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the TIMEWEAR Store,TIMEWEAR Commando Series Analog Digital Multif...,3.8 out of 5 stars,"8,240 ratings",-,10 Days Returns,"Wednesday, June 16",In stock.,Water Resistant | Note - please Do Not press a...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: FACHOI,Gold Square Fashion Quartz Casual Watch,-,-,-,10 Days Returns,"Thursday, June 17",In stock.,"Appearance: fashion, leisure, simple and trend...",https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Decode Store,Decode Matrix Analogue Men's Watch (Black Dial...,3.4 out of 5 stars,843 ratings,-,10 Days Returns,"Saturday, June 19",In stock.,Dial Color: Black; Dial Material: Rubber || ...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the OPTIMA Store,OPTIMA Analogue Brown Dial Watch,4.3 out of 5 stars,7 ratings,₹ 990.00,10 Days Returns,"Wednesday, June 16",In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...
4,Brand: Acnos,Acnos Brand - A Digital Watch with Square LED ...,4.3 out of 5 stars,63 ratings,₹ 399.00,10 Days Returns,"Wednesday, June 16",In stock.,Warranty type:Manufacturer; 1 Years Manufactur...,https://www.amazon.in/Acnos-Brand-Shockproof-M...
...,...,...,...,...,...,...,...,...,...,...
184,Visit the TYPIFY Store,TYPIFY ® Women's Black Multicolor Bracelet Rhi...,3.5 out of 5 stars,4 ratings,₹ 299.00,10 Days Returns,"Wednesday, June 16",In stock.,"Dial Color: White, Case Shape: Round, Dial Gla...",https://www.amazon.in/TYPIFY-Multicolor-Bracel...
185,Visit the Decode Store,Decode Analogue Men's Watch (Black Dial Black ...,3.6 out of 5 stars,140 ratings,-,7 Days Replacement,"Saturday, June 19",In stock.,Mesh strap || Day and date functioning || ...,https://www.amazon.in/gp/slredirect/picassoRed...
186,Visit the SWADESI STUFF Store,Swadesi Stuff Grey Dial Grey Leather Strap Ana...,5.0 out of 5 stars,1 rating,₹ 249.00,10 Days Returns,"Wednesday, June 16",In stock.,Dial Color: Grey | Band Color: Grey || Dial ...,https://www.amazon.in/gp/slredirect/picassoRed...
187,Visit the SWADESI STUFF Store,Swadesi Stuff Moon Analog Men & Women's Watch ...,2.5 out of 5 stars,5 ratings,₹ 349.00,10 Days Returns,"Wednesday, June 16",In stock.,Dial Color: Black | Band Color: Black || Dia...,https://www.amazon.in/gp/slredirect/picassoRed...


In [9]:
products.to_csv('Amazon.csv')

#### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [50]:
# Opening images.google.com
driver.get('https://images.google.com/')

In [51]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing fruits keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [52]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,20000)")

start scrolling to generate more images on the page...


In [53]:
img_urls = []
img_data = []
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

155

In [54]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("G:/Flip robo/images"+str(i)+".jpg", "wb")
    file.write(response.content)

In [61]:
#Extracting Cars images

In [55]:
search_bar = driver.find_element_by_xpath('//input[@id="REsRA"]')    # Finding the search bar using it's xpath
search_bar.clear()
search_bar.send_keys("cars")       # Inputing cars keyword to search rock images
search_button = driver.find_element_by_xpath('//button[@id="BIqFsb"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [58]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,20000)")

start scrolling to generate more images on the page...


In [59]:
img_urls = []
img_data = []
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

152

In [60]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("G:/Flip robo/images/cars"+str(i)+".jpg", "wb")
    file.write(response.content)

In [62]:
#Extracting Machine Learing images

In [63]:
search_bar = driver.find_element_by_xpath('//input[@id="REsRA"]')    # Finding the search bar using it's xpath
search_bar.clear()
search_bar.send_keys("Machine Learning")       # Inputing Machine Learning keyword to search rock images
search_button = driver.find_element_by_xpath('//button[@id="BIqFsb"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [64]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,100000)")

start scrolling to generate more images on the page...


In [65]:
img_urls = []
img_data = []
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

112

In [66]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("G:/Flip robo/images/cars/Machine learning"+str(i)+".jpg", "wb")
    file.write(response.content)

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [67]:
# Asking for user input
driver.get('https://www.flipkart.com/')

time.sleep(3)
product = input(" Enter the name of the mobile phone you want to search : ")
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(product)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

 Enter the name of the mobile phone you want to search : asus rog 3
No Login page


In [68]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [69]:
len(flip_urls)

12

In [70]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [71]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
    
                                                               

In [72]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

12 12 12 12 12


In [73]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Battery Type,Price,URL
0,ASUS,ROG Phone 3,Black,12 GB,128 GB,64MP + 13MP + 5MP,-,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 865+ (SM8250),Octa Core,6000 mAh,-,"₹45,999",https://www.flipkart.com/asus-rog-phone-3-blac...
1,ASUS,ROG Phone 3,Black,8 GB,128 GB,64MP + 13MP + 5MP,-,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 865+ (SM8250),Octa Core,6000 mAh,-,"₹41,999",https://www.flipkart.com/asus-rog-phone-3-blac...
2,ASUS,ROG Phone 3,Black,12 GB,256 GB,64MP + 13MP + 5MP,-,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 865+ (SM8250),Octa Core,6000 mAh,-,"₹54,999",https://www.flipkart.com/asus-rog-phone-3-blac...
3,ASUS,ROG Phone 5,White,8 GB,128 GB,64MP + 13MP + 5MP,24MP Front Camera,17.22 cm (6.78 inch),2448 x 1080 Pixels,Qualcomm Snapdragon 888 (SM8350),Octa Core,6000 mAh,-,"₹49,999",https://www.flipkart.com/asus-rog-phone-5-whit...
4,ASUS,ROG Phone 5,Black,8 GB,128 GB,64MP + 13MP + 5MP,24MP Front Camera,17.22 cm (6.78 inch),2448 x 1080 Pixels,Qualcomm Snapdragon 888 (SM8350),Octa Core,6000 mAh,-,"₹49,999",https://www.flipkart.com/asus-rog-phone-5-blac...
5,ASUS,ROG Phone 5,Black,12 GB,256 GB,64MP + 13MP + 5MP,24MP Front Camera,17.22 cm (6.78 inch),2448 x 1080 Pixels,Qualcomm Snapdragon 888 (SM8350),Octa Core,6000 mAh,-,"₹57,999",https://www.flipkart.com/asus-rog-phone-5-blac...
6,ASUS,ROG Phone II,Black,8 GB,128 GB,48MP + 13MP,-,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 855 Plus,Octa Core,6000 mAh,Li-Po,"₹40,999",https://www.flipkart.com/asus-rog-phone-ii-bla...
7,ASUS,ROG Phone II,Black,12 GB,512 GB,48MP + 13MP,-,16.74 cm (6.59 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 855 Plus,Octa Core,6000 mAh,Li-Po,"₹62,999",https://www.flipkart.com/asus-rog-phone-ii-bla...
8,ASUS,ROG,Black,8 GB,128 GB,12MP + 8MP,8MP Front Camera,15.24 cm (6 inch),2160 x 1080 pixels,Qualcomm Speed-binned Snapdragon 845,Octa Core,4000 mAh,-,"₹26,499",https://www.flipkart.com/asus-rog-black-128-gb...
9,ASUS,ROG Phone 5 Ultimate,White,18 GB,512 GB,64MP + 13MP + 5MP,24MP Front Camera,17.22 cm (6.78 inch),2448 x 1080 Pixels,Qualcomm Snapdragon 888 (SM8350),Octa Core,6000 mAh,-,"₹79,999",https://www.flipkart.com/asus-rog-phone-5-ulti...


#### Q5. 5.	Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [74]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Hyderabad
URL Extracted:  https://www.google.co.in/maps/place/Hyderabad,+Telangana/@17.4121531,78.1278362,10z/data=!3m1!4b1!4m5!3m4!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671
Latitude = 17.4121531, Longitude = 78.1278362


#### Q6.	Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [75]:
driver.get('https://trak.in/')

In [76]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [77]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [78]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [79]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [80]:
driver.get('https://www.digit.in/')

In [81]:
#clickng on top 10 option 
top_10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]/a")
top_10.click()

#clicking on laptops option
laptops=driver.find_element_by_xpath("//div[@class='categoty_list']/button[2]")
laptops.click()

#best gaming laptops link
best_gaming=driver.find_element_by_xpath("//div[@id='laptops']/div[3]/a")
driver.get(best_gaming.get_attribute('href'))

#intialising lists
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")#list of specificaion name
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")#values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,Price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,ALIENWARE AREA 51M R2,No details available,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,ALIENWARE M15 R3,"341,990",WINDOWS 10 HOME,"15.6"" (3840 X 2160)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I9-10980HK | NA,NA,NA,NA
2,ASUS ROG STRIX SCAR 15,No details available,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD RYZEN™ 9 5900HX | 3.3 GHZ,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,ASUS ROG ZEPHYRUS G14,"164,990",WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION 5I,"71,990",WINDOWS 10 PRO,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,ASUS ROG ZEPHYRUS DUO 15,"199,990",WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
6,ACER ASPIRE 7 GAMING,"56,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,8 GBGB DDR4,AMD RYZEN™ 5-5500U HEXA-CORE | NA,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650


In [82]:
df.to_csv("Best_Gaming_laptops_digit.csv")